**Q1:**

*A:* In the covariance equation from the numerator we learn the direction of related changes of the variables for it sums the dot products of the difference of every sample from each dataset from it's mean. For example as one variable increases and the other one decreases most of the products that will be summed up will be negative and as a result we'll get a negative covariance. Because the magnitude of the covariance depends on the specific data set and the magnitude of the values in the sample it can't give us a unified measure for all data sets to measure how much the variables are "changing together". This correction comes with the correlation equation where the denominator standartizes the values of the dataset and thus gives us a unified measure for all possible data sets not only for the direction of mutual change but also it's magnitude.

## add the equations of cov and corr here

*B:* 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_iris
import seaborn as sns
import plotly.express as px

In [6]:
Iris_df = pd.read_csv("iris.csv")
print(Iris_df.head())

   sepal.length  sepal.width  petal.length  petal.width variety
0           5.1          3.5           1.4          0.2  Setosa
1           4.9          3.0           1.4          0.2  Setosa
2           4.7          3.2           1.3          0.2  Setosa
3           4.6          3.1           1.5          0.2  Setosa
4           5.0          3.6           1.4          0.2  Setosa


## add visualization 

*C:*

In [14]:
Iris_df_numbers = Iris_df.select_dtypes(include=['number'])

In [9]:
def calcCov(x, y):
  x = x.to_numpy()
  y = y.to_numpy()
  x_normalized = np.apply_along_axis(lambda x: x - np.mean(x), 0, x)
  y_normalized = np.apply_along_axis(lambda y: y - np.mean(y), 0, y)
  return np.dot(x_normalized.T, y_normalized) / (x.shape[0]-1) # for a covariance from a sample and not population

In [11]:
def covMat(data):
  cov_mat = np.zeros((data.shape[1], data.shape[1]))
  for i in range(data.shape[1]):
    for j in range(data.shape[1]):
      cov_mat[i, j] = calcCov(data.iloc[:, i], data.iloc[:, j])
  return cov_mat

*D: test 1*

In [20]:
print(covMat(Iris_df_numbers))
print(np.cov(Iris_df_numbers, rowvar=False)) # rowvar = False, cause in our dataframe each column represents a variable and not the rows.

[[ 0.68569351 -0.042434    1.27431544  0.51627069]
 [-0.042434    0.18997942 -0.32965638 -0.12163937]
 [ 1.27431544 -0.32965638  3.11627785  1.2956094 ]
 [ 0.51627069 -0.12163937  1.2956094   0.58100626]]
[[ 0.68569351 -0.042434    1.27431544  0.51627069]
 [-0.042434    0.18997942 -0.32965638 -0.12163937]
 [ 1.27431544 -0.32965638  3.11627785  1.2956094 ]
 [ 0.51627069 -0.12163937  1.2956094   0.58100626]]


*E:*

In [12]:
def corrMat(data):
  cov_mat = covMat(data)
  denomenator = np.zeros((data.shape[1], data.shape[1]))
  corr_mat = np.zeros((data.shape[1], data.shape[1]))
  for i in range(data.shape[1]):
    for j in range(data.shape[1]):
      denomenator[i,j] = np.sqrt(data.iloc[:, i].var() * data.iloc[:, j].var())
      corr_mat[i,j] = cov_mat[i,j] / denomenator[i,j]
  return corr_mat

In [19]:
print(corrMat(Iris_df_numbers))
print(np.corrcoef(Iris_df_numbers, rowvar=False))

[[ 1.         -0.11756978  0.87175378  0.81794113]
 [-0.11756978  1.         -0.4284401  -0.36612593]
 [ 0.87175378 -0.4284401   1.          0.96286543]
 [ 0.81794113 -0.36612593  0.96286543  1.        ]]
[[ 1.         -0.11756978  0.87175378  0.81794113]
 [-0.11756978  1.         -0.4284401  -0.36612593]
 [ 0.87175378 -0.4284401   1.          0.96286543]
 [ 0.81794113 -0.36612593  0.96286543  1.        ]]


## G. add heatmaps to the code 

**Q2:**

In [18]:
laptop_price_df = pd.read_csv("laptop_price - dataset.csv")
print(laptop_price_df.head())

  Company      Product   TypeName  Inches                    ScreenResolution  \
0   Apple  MacBook Pro  Ultrabook    13.3  IPS Panel Retina Display 2560x1600   
1   Apple  Macbook Air  Ultrabook    13.3                            1440x900   
2      HP       250 G6   Notebook    15.6                   Full HD 1920x1080   
3   Apple  MacBook Pro  Ultrabook    15.4  IPS Panel Retina Display 2880x1800   
4   Apple  MacBook Pro  Ultrabook    13.3  IPS Panel Retina Display 2560x1600   

  CPU_Company       CPU_Type  CPU_Frequency (GHz)  RAM (GB)  \
0       Intel        Core i5                  2.3         8   
1       Intel        Core i5                  1.8         8   
2       Intel  Core i5 7200U                  2.5         8   
3       Intel        Core i7                  2.7        16   
4       Intel        Core i5                  3.1         8   

                Memory GPU_Company                GPU_Type  OpSys  \
0            128GB SSD       Intel  Iris Plus Graphics 640  macOS

In [ ]:
plt.figure(figsize=(16, 13))
sns.boxplot(laptop_price_df, x='Company', y='Price (Euro)', hue="Company")
plt.title("Distribution and Variation of Laptop Prices Across Companies")

In [ ]:
laptop_mean_price_df = laptop_price_df[['Price (Euro)','Company']].groupby('Company').mean() #need to chane the title to mean price
laptop_mean_price_df.rename(columns={'Price (Euro)':'Mean Laptop Price (Euro)'}, inplace=True)
laptop_mean_price_df

In [ ]:
laptop_price_company = laptop_price_df[['Price (Euro)','Company']]
laptop_price_company_max = laptop_price_company[laptop_price_company['Price (Euro)']==laptop_price_company.groupby('Company').max().max()['Price (Euro)']]
laptop_price_company_max.rename(columns={'Price (Euro)':'Max Laptop Price (Euro)'}, inplace=True)
laptop_price_company_max